# This notebook contains your standard data exploration

In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import re

In [8]:
data = pd.read_csv('./scrapers/asuntojen-hintatiedot/scraped_data/2021-04-16.csv')

In [9]:
data

,neighborhood,room_arrangement,house_type,square_meters,price_including_loans,price_per_square_meters,built_in,floor,has_elevator,shape,lot,energy_classification
0,Kivenlahti,AH,apartment,12.0,20000,1667,2020,1/3,True,good,rented,B-2018
1,Tapiola,"1h,kt,lasitettu...",apartment,35.0,308000,8800,2021,6/10,True,good,owned,B
2,Tapiola,"1h,kt.lasitettu...",apartment,35.0,299950,8570,2021,5/10,True,good,owned,B
3,Tapiola,"1h,kt,lasitettu...",apartment,35.0,316750,9050,2021,7/10,True,good,owned,B
4,Tapiola,"1h,kt,lasitettu...",apartment,35.0,325500,9300,2021,8/10,True,good,owned,B
...,...,...,...,...,...,...,...,...,...,...,...,...
1569,NaN,4h+k+khh+s+aula...,townhouse,169.0,420000,2485,1989,1/2,False,satisfactory,owned,D-2018
1570,NaN,"4h,k,khh,sauna ...",townhouse,79.0,425000,5380,2004,1/2,False,good,owned,NaN
1571,NaN,"3-4h, k ja saun...",townhouse,150.0,365000,2433,1952,1/3,False,good,owned,D-2018
1572,NaN,"5-6 h, k, s, kp...",townhouse,137.5,412000,2996,2000,1/2,False,satisfactory,owned,E-2013


In [40]:
def get_number_of_rooms(data):
    room_arrangement = data["room_arrangement"]
    if not type(room_arrangement) == str:
        return 0
    if (room_arrangement == 'AH'):
        # AH = asuinhuoneisto = single room, no toilet/kitchen/etc, use special value
        return 0.5
    variable_rooms_regexp = re.compile('([0-9])-([0-9])\s?h', re.IGNORECASE)
    variable_rooms = variable_rooms_regexp.findall(room_arrangement)
    if len(variable_rooms) > 0:
        return ((int(variable_rooms[0][0]) + int(variable_rooms[0][1])) / 2)
    simple_rooms_regexp = re.compile('([0-9])\s?h', re.IGNORECASE)
    simple_rooms = simple_rooms_regexp.findall(room_arrangement)
    if len(simple_rooms) > 0:
        return int(simple_rooms[0])
    return room_arrangement

In [41]:
data["number_of_rooms"] = data.apply(get_number_of_rooms, axis=1)

In [42]:
data

,neighborhood,room_arrangement,house_type,square_meters,price_including_loans,price_per_square_meters,built_in,floor,has_elevator,shape,lot,energy_classification,number_of_rooms
0,Kivenlahti,AH,apartment,12.0,20000,1667,2020,1/3,True,good,rented,B-2018,0.5
1,Tapiola,"1h,kt,lasitettu...",apartment,35.0,308000,8800,2021,6/10,True,good,owned,B,1
2,Tapiola,"1h,kt.lasitettu...",apartment,35.0,299950,8570,2021,5/10,True,good,owned,B,1
3,Tapiola,"1h,kt,lasitettu...",apartment,35.0,316750,9050,2021,7/10,True,good,owned,B,1
4,Tapiola,"1h,kt,lasitettu...",apartment,35.0,325500,9300,2021,8/10,True,good,owned,B,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,NaN,4h+k+khh+s+aula...,townhouse,169.0,420000,2485,1989,1/2,False,satisfactory,owned,D-2018,4
1570,NaN,"4h,k,khh,sauna ...",townhouse,79.0,425000,5380,2004,1/2,False,good,owned,NaN,4
1571,NaN,"3-4h, k ja saun...",townhouse,150.0,365000,2433,1952,1/3,False,good,owned,D-2018,3.5
1572,NaN,"5-6 h, k, s, kp...",townhouse,137.5,412000,2996,2000,1/2,False,satisfactory,owned,E-2013,5.5
